<a href="https://colab.research.google.com/github/davidfague/Stylized-ReducedOrder-L5-Model/blob/main/SROL5_NR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get SROL5 Model and Neuron_reduce

In [1]:
NR=False # set True to use original neuron_reduce, False for test_neuron_reduce

In [2]:
!git clone https://github.com/davidfague/Stylized-ReducedOrder-L5-Model.git

%cd Stylized-ReducedOrder-L5-Model/

!pip install neuron
from neuron import h,nrn,gui
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import time
from io import StringIO
import os, sys
from typing import Optional, List, Tuple, Union
from functools import partial
import scipy.signal as ss
import scipy.stats as st
import h5py
import matplotlib.pyplot as plt
import math
from utils.plot_results import plot_LFP_traces,plot_LFP_heatmap
from utils.plot_morphology import plot_morphology
import re

from __future__ import division

if NR == True:
  !pip install neuron_reduce
  import neuron_reduce
  from neuron_reduce import subtree_reductor
else:
  # try to use the test_neuron_reduce from my github
  from test_neuron_reduce.subtree_reductor_func import subtree_reductor

fatal: destination path 'Stylized-ReducedOrder-L5-Model' already exists and is not an empty directory.
/content/Stylized-ReducedOrder-L5-Model
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#refresh github
import shutil
%cd ..
shutil.rmtree('Stylized-ReducedOrder-L5-Model/') #removes folder
!git clone https://github.com/davidfague/Stylized-ReducedOrder-L5-Model.git
%cd Stylized-ReducedOrder-L5-Model/

/content
Cloning into 'Stylized-ReducedOrder-L5-Model'...
remote: Enumerating objects: 1787, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 1787 (delta 46), reused 0 (delta 0), pack-reused 1700
Receiving objects: 100% (1787/1787), 113.67 MiB | 19.34 MiB/s, done.
Resolving deltas: 100% (904/904), done.
/content/Stylized-ReducedOrder-L5-Model


In [4]:
mechanisms_path = 'mechanisms'  # need to compile the folder first
geometry_file = 'geom_parameters.csv'
# geometry_file = 'geom_NR_test.csv' # test simple geometry

tstop = 1000  # ms simulation duration
dt = 0.1  # ms. does not allow change

h.load_file('stdrun.hoc')
#h.nrn_load_dll(os.path.join(mechanisms_path,'nrnmech.dll'))  # running this line more than once might crash the kernel
h.nrn_load_dll(os.path.join(mechanisms_path,'x86_64/.libs/libnrnmech.so'))  # use this line instead in Linux
geo_standard = pd.read_csv(geometry_file,index_col='id')
h.tstop = tstop
h.dt = dt
h.steps_per_ms = 1/h.dt

#pick a random seed to initalize random values
import random
random.seed(427)

In [5]:
from SROL5.build_model import Simulation
# from stylized_module.ecp import EcpMod, newposition
# from stylized_module.recorder import Recorder
# from stylized_module.point_current import Point_current

#Define Cell Orientation, Biophysics, and Geometry

In [6]:
x = np.zeros(96)
y = np.linspace(-1900,1900,96)
z = np.zeros(96)
elec_pos = pos = np.column_stack((x,y,z))

#choose orientation of neuron
loc_param = [50,0,10,0,1,0]  # example position #D: initially this was [0,0,50,0.2,0.75,1.5], (x,y,z,theta,h,phi), h=[-1,1] phi=[-pi,pi] h=.7071067812 for 90 deg
#note on parameters: Rotate(alpha,h,phi): first rotate alpha about y-axis (spin), then rotate arccos(h) about x-axis (elevation), then rotate phi about y axis (azimuth).


# change geo_parameters from their default values #************************NOTE**********************: doesn't change the geometry values used in geom_iterrows & sec['parameter'] (like when synapses are adding using geom);
geo_param = [ # -1 uses default value                                                                  #better to edit the geom file to guarantee edits until the automation is fixed
            #soma
            -1,  # radius
#proximal trunk
            -1, # excitation ('TRUE'/'FALSE')
            -1, # inhibition ('TRUE'/'FALSE')
            -1, # length
            -1, # radius
            -1, # exc FR mean
            -1, # exc FR std
            -1, # inh FR mean
            -1, # inh FR std
            -1, # exc P_0 mean (0 to 1)
            -1, # exc P_0 std
            -1, # inh P_0 mean (0 to 1)
            -1, # inh P_0 std
#middle trunk
            -1, # excitation ('TRUE'/'FALSE')
            -1, # inhibition ('TRUE'/'FALSE')
            -1, # length
            -1, # radius
            -1, # exc FR mean
            -1, # exc FR std
            -1, # inh FR mean
            -1, # inh FR std
            -1, # exc P_0 mean (0 to 1)
            -1, # exc P_0 std
            -1, # inh P_0 mean (0 to 1)
            -1, # inh P_0 std
#distal trunk
            -1, # excitation ('TRUE'/'FALSE')
            -1, # inhibition ('TRUE'/'FALSE')
            -1, # length
            -1, # radius
            -1, # exc FR mean
            -1, # exc FR std
            -1, # inh FR mean
            -1, # inh FR std
            -1, # exc P_0 mean (0 to 1)
            -1, # exc P_0 std
            -1, # inh P_0 mean (0 to 1)
            -1, # inh P_0 std
#basal dendrites
            -1, # excitation ('TRUE'/'FALSE')
            -1, # inhibition ('TRUE'/'FALSE')
            -1, # number of dendrites
            -1, # proximal section length
            -1, # middle section length
            -1, # distal section length
            -1, # proximal radius
            -1, # middle radius
            -1, # distal radius
            -1, # angle
            -1, # exc FR mean
            -1, # exc FR std
            -1, # inh FR mean
            -1, # inh FR std
            -1, # exc P_0 mean (0 to 1)
            -1, # exc P_0 std
            -1, # inh P_0 mean (0 to 1)
            -1, # inh P_0 std
#oblique dendrites
            -1, # excitation ('TRUE'/'FALSE')
            -1, # inhibition ('TRUE'/'FALSE')
            -1, # number of dendrites # counted 15 in hay model
            -1, # length
            -1, # radius
            -1, # angle
            -1, # exc FR mean
            -1, # exc FR std
            -1, # inh FR mean
            -1, # inh FR std
            -1, # exc P_0 mean (0 to 1)
            -1, # exc P_0 std
            -1, # inh P_0 mean (0 to 1)
            -1, # inh P_0 std
#tuft dendrites
            -1, # excitation ('TRUE'/'FALSE')
            -1, # inhibition ('TRUE'/'FALSE')
            -1, # number of dendrites
            -1, # proximal section length
            -1, # middle section length
            -1, # distal section length
            -1, # proximal radius
            -1, # middle radius
            -1, # distal radius
            -1, # angle
            -1, # exc FR mean
            -1, # exc FR std
            -1, # inh FR mean
            -1, # inh FR std
            -1, # exc P_0 mean (0 to 1)
            -1, # exc P_0 std
            -1, # inh P_0 mean (0 to 1)
            -1, # inh P_0 std
#axon
            -1, # length
            -1, # radius # hay uses 0.5 #we had at 8
#passive basal
            -1, # number of dendrites
            -1, # length
            -1 # radius
        ]
      
      
# example of changing biophysical parameters (check "biophys_entries" in "Cell" class for meaning, bounds can also be found there)
#latest conductances from Detailed Model Updated 8/21/22
#Ih may be off for apical sections; The detailed model axon shas only g_pas, but we have given the axon some conductances closer to the soma, and may need to change
#also setting a conductance to 0.0 may throw some sort of error
biophys = [0.0000338,0.0000467,0.0000589,0.0000589,0.0000589,0.0000325,0.0000467, # g_pas of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #passive basal was 0.00001
                                    2.04,0.0213,0.0213,0.0213,0.0213,0.0,0.0, # gNaTa_t of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal # axon was 2.89618
                                    0.693,0.000261,0.000261,0.000261,0.000261,0.0,0.0, # gSKv3_1 of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #axon was 0.473799
           
                                    0.000992,0.0,0.0000555,(0.000555),.0000555,0.0,0.0,  # gCA_HVA of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #basal was 0.000992
                                    0.00343,0.0,0.000187,(0.0187),0.000187,0.0,0.0, # gCA_LVAst of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.0000675,0.0000675,(0.0000675),0.0000675,0.0, # gIm of basal, midTrunk, distTrunk, tuft, axon
           
                                    460.0,122,122,122,122,277.300774,122, # decay_CaDynamics of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.000501,0.000509,0.000509,0.000509,0.000509,0.000525,0.000509, # gamma_CaDynamics of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    # 0.0002,0.0002,0.0002,0.0002,0.0002,0.0001,0.0002, # gIh of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.0002,0.0002,0.0009596,0.0029005,.006997745,0.0001,0.0002, # gIh of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #calculated from distribute_channels #if sectioning tuft do disttuft:0.0118268; midtuft:.006997745; proxtuft:.0047005526
                                    0.0441,0.0012,0.0012,0.0012,0.0012,0.000047, # gSk_E2 of soma, basal, midTrunk, distTrunk, tuft, axon
                                    0.0812,0.0, # gK_Tst of soma, axon #axon was 0.077274
                                    0.00223,0.0, # gK_Pst of soma, axon #axon was 0.188851
                                    0.00172,0.0  # gNap_Et2 of soma, axon
                                    ]

#Define Simulation

In [7]:
timestart=time.time()
sim = Simulation(geo_standard,elec_pos,loc_param,geo_param=geo_param,biophys=biophys,gmax=0.05,scale=100.,spike_threshold = 10, synaptic_input = True, RunType = 'Full Model')  # 0.001 -0.012 #gmax=.005, scale=100
timestop=time.time()
elapseddeftime=timestop-timestart
simtime=tstop/1000 #convert from ms to s
print('It took',round(elapseddeftime),'sec to define the simulation model.') #need to optomize input grouping # location check takes a long time since it iterates through checking a list of synapse locations for each group/cluster within the section

now attempting sec_id: 1
allsections: [soma[0]] | pid[i]: 0 | psec: soma[0]
allsections: [soma[0], proxbasal[0].dend[0]] | pid[i]: 0 | psec: soma[0]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1]] | pid[i]: 0 | psec: soma[0]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2]] | pid[i]: 0 | psec: soma[0]
now attempting sec_id: 2
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxbasal[3].dend[3]] | pid[i]: 1 | psec: proxbasal[0].dend[0]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxbasal[3].dend[3], midbasal[0].dend[4]] | pid[i]: 2 | psec: proxbasal[1].dend[1]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxbasal[3].dend[3], midbasal[0].dend[4], midbasal[1].dend[5]] | pid[i]: 3 | psec: proxbasal[2].dend[2]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxba

#Testing neuron_reduce on stylized cell

In [8]:
# #check extraction of section type
# for section in sim.cells[0].apical:
#     full_sec_name = h.secname(sec=section)
#     print(full_sec_name)
#     sec_name_as_list = full_sec_name.split(".")
#     print(sec_name_as_list)
#     short_sec_name = sec_name_as_list[len(sec_name_as_list) - 1]
#     print(short_sec_name)
#     section_type = short_sec_name.split("[")[0]
#     print(section_type)
#     section_num = re.findall(r'\d+', short_sec_name)[0]

In [9]:
# #check extraction of section number
# for section in sim.cells[0].all:
#     sec_name = h.secname(sec=section)
#     print(sec_name)
#     ints_in_name = re.findall(r'\d+', sec_name)
#     sec_num = ints_in_name[len(ints_in_name) - 1]  # extracts section number
#     print(sec_num)

In [10]:
complex_cell=sim.cells[0]

In [11]:
synapses_list=[]
netcons_list=[]
for synapse in complex_cell.injection:
  synapses_list.append(synapse.pp_obj)
  netcons_list.append(synapse.nc)

In [12]:
# print(dir(synapses_list[0]))

In [13]:
type(complex_cell.soma)

list

In [14]:
print(complex_cell.soma[0].hname())

soma[0]


In [15]:
print(complex_cell.soma[0].hname()[-1] == ']')

True


In [16]:
# soma = complex_cell.soma[0] if complex_cell.soma.hname()[-1] == ']' else complex_cell.soma
# print(soma)

In [17]:
complex_cell=sim.cells[0]
# print(netcons_list)
#apply Neuron_Reduce to simplify the cell
reduced_cell, synapses_list, netcons_list = subtree_reductor(complex_cell, synapses_list, netcons_list, reduction_frequency=0, total_segments_manual=-1)

AttributeError: ignored